In [7]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from cnn_model_4 import EmotionRecognition

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionRecognition().to(device)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()

# Emotion labels
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Image transform
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
window = tk.Tk()
window.title("Emotion Recognition")
window.geometry("400x500")

label = tk.Label(window, text="Upload a face image", font=("Arial", 14))
label.pack(pady=10)

canvas = tk.Canvas(window, width=200, height=200)
canvas.pack()

result_label = tk.Label(window, text="", font=("Arial", 16))
result_label.pack(pady=20)

In [10]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    img = Image.open(file_path).convert("RGB")
    img.thumbnail((200, 200))
    tk_img = ImageTk.PhotoImage(img)
    canvas.create_image(100, 100, image=tk_img)
    canvas.image = tk_img

    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
        top_prob, top_class = torch.max(probs, 1)

    emotion = class_names[top_class.item()]
    result_label.config(text=f"Predicted Emotion: {emotion}")

In [11]:
button = tk.Button(window, text="Upload Image", command=upload_image)
button.pack(pady=10)

window.mainloop()